# Comparing h3 cells to administrative divisions

While most of our work has focused on the consistent, global, hexagon grid, the ultimate goal of the S2S work program is to generate a database of geospatial aggregates at the administrative level 2. **We are still in the process of acquiring the admin bounds we are going to publish.**

This notebook focuses on identifying percentage overlap between our official administrative divisions and our gobal hexagonal grid.

In [2]:
import sys, os, multiprocessing
import h3ronpy, h3

import pandas as pd
import geopandas as gpd


from tqdm.notebook import tqdm
from h3ronpy.pandas.vector import geodataframe_to_cells, cells_dataframe_to_geodataframe
from h3ronpy import ContainmentMode

from GOSTrocks.misc import tPrint
import GOSTrocks.mapMisc as mapMisc


In [3]:
base_folder = r"C:\WBG\Work\S2S"
admin_folder = r"C:\WBG\Work\data\ADMIN\BETTER_FORMATS\simplified_output"
out_folder = os.path.join(base_folder, "ADM_RESULTS")

if not os.path.exists(out_folder):
    os.makedirs(out_folder)

# Get the admin boundaries
admin_id = 'ADM2CD_c'
admin_file = os.path.join(admin_folder, "WB_GAD_adm2_simplified.gpkg")
in_admin = gpd.read_file(admin_file)
in_admin.head()

,ISO_A3,ISO_A2,WB_A3,WB_REGION,WB_STATUS,NAM_0,NAM_1,ADM1CD_c,GEOM_SRCE,NAM_2,ADM2CD_c,geometry
0,CHN,CN,CHN,EAP,Member State,China,Anhui Sheng,CHN001,WB GAD,Hefei,CHN001006,"MULTIPOLYGON (((117.85668 32.02414, 117.76963 ..."
1,CHN,CN,CHN,EAP,Member State,China,Anhui Sheng,CHN001,WB GAD,Bengbu,CHN001002,"MULTIPOLYGON (((117.94735 33.3384, 117.95826 3..."
2,CHN,CN,CHN,EAP,Member State,China,Anhui Sheng,CHN001,WB GAD,Huainan,CHN001008,"MULTIPOLYGON (((117.20039 32.63284, 117.11077 ..."
3,CHN,CN,CHN,EAP,Member State,China,Anhui Sheng,CHN001,WB GAD,Huaibei,CHN001007,"MULTIPOLYGON (((116.88567 33.37158, 116.85864 ..."
4,CHN,CN,CHN,EAP,Member State,China,Anhui Sheng,CHN001,WB GAD,Name Unknown,CHN001012,"MULTIPOLYGON (((116.80599 33.30873, 116.81557 ..."


In [4]:
def run_country_mp(cntry_label, in_data, out_folder, admin_id):
    """ Iterate through the country and get intersecting hex_ids for each feature in in_data

    Parameters
    ----------
    cntry_label : string
        The country label to use for the output
    in_data : geopandas.GeoDataFrame
        The input data to process of admini features
    out_folder : string
        The output folder to save the results
    """    
    out_file = os.path.join(out_folder, f'{cntry_label}_h3_intersect.csv')
    if not os.path.exists(out_file):    
        all_res = []
        for idx, row in tqdm(in_data.iterrows(), total=in_data.shape[0], desc=f'Processing {cntry_label}'):
            in_shape = gpd.GeoDataFrame(row.to_frame().T, geometry="geometry", crs=in_data.crs)
            in_shape['geometry'] = in_shape['geometry'].apply(lambda x: x.buffer(0))                                        
            cur_res = get_bounds(in_shape, admin_id, 6)
            cur_res['cntry'] = cntry_label
            all_res.append(cur_res)
        
        country_res = pd.concat(all_res, ignore_index=True)
        country_res.to_csv(out_file, index=False)
    else:
        country_res = pd.read_csv(out_file)
    return(country_res)


def get_bounds(in_shp, gID, h3_lvl=6):
    """ Generate a geodataframe for the supplied in_shp with the H3 cells and % overlap

    Parameters
    ----------
    in_shp : shapely.geometry.polygon.Polygon
        The input shapely polygon
    gID : string
        The column name to use for the ID in the output
    h3_lvl : int
        The H3 level to use for the hexagons, default is 6
    """
    # extract the H3 cells
    cols_to_keep = [gID, 'cell', 'overlap']
    cell_ax = cells_dataframe_to_geodataframe(geodataframe_to_cells(in_shp, 6, ContainmentMode.IntersectsBoundary))
    cell_ax['cell'] = cell_ax['cell'].apply(lambda x: hex(x)[2:])    
    # Identify contained and overlapping hexes with the admin bounds
    contained_h3 = cell_ax.sjoin(in_shp, predicate='within')
    missed_h3 = cell_ax[~cell_ax['cell'].isin(contained_h3['cell'])]
    # calculate h3x overlap with feature
    shp_area = in_shp.union_all()
    cell_ax['overlap'] = 0.0
    cell_ax.loc[contained_h3.index, 'overlap'] = 1.0
    cell_ax.loc[missed_h3.index, 'overlap'] = cell_ax.loc[missed_h3.index,'geometry'].apply(lambda x: x.intersection(shp_area).area/x.area)\
    
    return cell_ax.loc[:, cols_to_keep].reset_index(drop=True)


sel_admin = in_admin.loc[in_admin['WB_A3'] == 'KEN']
xx = get_bounds(gpd.GeoDataFrame(sel_admin.iloc[0].to_frame().T, geometry="geometry", crs=sel_admin.crs), admin_id, 6)
xx.head(10)



,ADM2CD_c,cell,overlap
0,KEN001006,867a41217ffffff,0.232826
1,KEN001006,867a4121fffffff,0.229676
2,KEN001006,867a41287ffffff,1.000000
3,KEN001006,867a4128fffffff,1.000000
4,KEN001006,867a41297ffffff,1.000000
5,KEN001006,867a4129fffffff,1.000000
6,KEN001006,867a412a7ffffff,0.943661
7,KEN001006,867a412afffffff,1.000000
8,KEN001006,867a412b7ffffff,1.000000
9,KEN001006,867a412c7ffffff,0.982841


In [5]:
mp_args = []
for cntry, data in in_admin.groupby('WB_A3'):
    mp_args.append([cntry, data, out_folder, admin_id])
''' This isn't working 
# Run multiprocessing
tPrint("Starting multiprocessing")
with multiprocessing.Pool(processes=multiprocessing.cpu_count() - 1) as pool:
    results = list(tqdm(pool.imap(run_country_mp, mp_args), total=len(mp_args), desc='Processing countries'))

'''

' This isn\'t working \n# Run multiprocessing\ntPrint("Starting multiprocessing")\nwith multiprocessing.Pool(processes=multiprocessing.cpu_count() - 1) as pool:\n    results = list(tqdm(pool.imap(run_country_mp, mp_args), total=len(mp_args), desc=\'Processing countries\'))\n\n'

In [6]:
for cntry, data, outFolder, adminID in tqdm(mp_args):
    print(f'Processing {cntry}')
    if not cntry in ['FJI', 'RUS', 'MLT', 'VEN']: # 'FJI','RUS There are topological errors current dataset
        res = run_country_mp(cntry, data, outFolder, admin_id)

  0%|          | 0/235 [00:00<?, ?it/s]

Processing ABW


Processing ABW:   0%|          | 0/1 [00:00<?, ?it/s]

Processing ADO


Processing ADO:   0%|          | 0/7 [00:00<?, ?it/s]

...

Processing ZAF:   0%|          | 0/52 [00:00<?, ?it/s]

Processing ZMB


Processing ZMB:   0%|          | 0/116 [00:00<?, ?it/s]

Processing ZWE


Processing ZWE:   0%|          | 0/63 [00:00<?, ?it/s]

# DEBNRURGGIGN

In [ ]:
shape(h3.cells_to_geo([c['cell']])).__class__

In [ ]:
sel_bounds = get_bounds(in_data.loc[in_data['NAM_2'] == "Annapolis"], admin_id)
h3_geo = []

for idx, c in sel_bounds.iterrows():
    try:
        h3_geo.append(shape(h3.cells_to_geo([c['cell']])))
    except:
        print(f'Error in {c["cell"]}')
        continue

sel_bounds['geometry'] = h3_geo
sel_bounds = gpd.GeoDataFrame(sel_bounds, geometry='geometry', crs=4326)

In [ ]:
mapMisc.static_map_vector(sel_bounds, map_column = 'overlap')

In [ ]:
sel_bounds.to_parquet(os.path.join(out_folder, f'{sel_iso3}_h3_intersect.parquet'))

In [ ]:
dir(h3)

In [ ]:
cntry_label = 'RUS'
in_data = in_admin.loc[in_admin['WB_A3'] == cntry_label]
in_data.plot()

In [ ]:
in_data.to_file(os.path.join(out_folder, f'{cntry_label}_h3_intersect.gpkg'), driver='GPKG')

In [ ]:
all_res = []
for idx, row in in_data.iterrows():
    tPrint(idx)
    in_shape = gpd.GeoDataFrame(row.to_frame().T, geometry="geometry", crs=in_data.crs)
    in_shape['geometry'] = in_shape['geometry'].apply(lambda x: x.buffer(0))                                        
    cur_res = get_bounds(in_shape, 6)
    cur_res['cntry'] = cntry_label
    all_res.append(cur_res)

In [ ]:
sel_row = in_data.loc[24588]
in_shp = gpd.GeoDataFrame(sel_row.to_frame().T, geometry="geometry", crs=in_data.crs)
            

In [ ]:
cell_ax

In [ ]:
cell_ax.to_file(os.path.join(out_folder, f'{cntry_label}_h3_intersect.gpkg'), driver='GPKG')

In [ ]:
tPrint('Starting')
cols_to_keep = ['gID', 'cell', 'overlap']
cell_ax = cells_dataframe_to_geodataframe(geodataframe_to_cells(in_shp, 6, ContainmentMode.IntersectsBoundary))
cell_ax['cell'] = cell_ax['cell'].apply(lambda x: hex(x)[2:])
tPrint("Generated H3 cells")
shp_area = in_shp.union_all()
cell_ax['overlap'] = cell_ax['geometry'].apply(lambda x: x.intersection(shp_area).area/x.area)
tPrint("Calculated overlap")
    

# return cell_ax.loc[:, cols_to_keep].reset_index(drop=True)

In [ ]:
x.area